## Experiment 12: 1-Dimensional Convolutional Neural Network (1D-CNN) Model with BERT Embeddings

#### (Version 1)


**Install the correct versions of tensorflow-text and tf-models-official**

In [1]:
!pip install -U "tensorflow-text==2.13.*"
!pip install "tf-models-official==2.13.*"

**Import the necessary libraries**

In [2]:
import pandas as pd
import numpy as np
import time

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import tensorflow_hub as hub
import tensorflow_text as text  # Required for BERT preprocessing
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.metrics import ConfusionMatrixDisplay

----

**1. Load in the data**

In [3]:
file_path = "top_and_non_golf_course_reviews.csv"
df = pd.read_csv(file_path)

# Create a new label column that indicates whether the review is a top100 course or not
df['top100'] = df['label'].apply(lambda x: 1 if x == 'top100' else 0)

----
**2. Split the dataset into training, validation, and testing sets**

In [4]:
train_df, remaining = train_test_split(df, test_size=0.33, stratify=df['top100'], random_state=42)
val_df, test_df = train_test_split(remaining, test_size=0.5, stratify=remaining['top100'], random_state=42)

# Check the shape of the training, validation, and test sets
print(f"Training Dataset Shape: {train_df.shape}")
print(f"Validation Dataset Shape: {val_df.shape}")
print(f"Test Dataset Shape: {test_df.shape}")

Training Dataset Shape: (80, 11)
Validation Dataset Shape: (20, 11)
Test Dataset Shape: (20, 11)


----
**3. Convert the split Pandas DataFrames into TensorFlow Datasets**

DONT ACTUALLY NEED TO DO THIS WITH THE PREPARE_DATASET FUNCTION CREATED IN STEP 6

In [5]:
# train_ds = tf.data.Dataset.from_tensor_slices(dict(train_df))
# val_ds = tf.data.Dataset.from_tensor_slices(dict(val_df))
# test_ds = tf.data.Dataset.from_tensor_slices(dict(test_df))

----
**4. Load in the BERT Models**

In [6]:
bert_preprocess_model = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3")
bert_encoder = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/4")

----
**5. Create a function to get BERT embeddings**

In [7]:
def get_bert_embeddings(text_input):
    preprocessed = bert_preprocess_model(text_input)
    return bert_encoder(preprocessed)['sequence_output']

----
**6. Define a function that prepares the dataset for training**

In [8]:
def prepare_dataset(df, batch_size=32):
    """
    Prepare the dataset for training by creating a TensorFlow dataset from the input DataFrame.

    Parameters:
    - df: The input DataFrame containing 'review_text' and 'top100' columns.
    - batch_size: The batch size for the dataset (default value is 32).

    Returns:
    - dataset: A TensorFlow dataset with the 'review_text' and 'top100' columns batched and mapped using BERT embeddings.
    - prefetch: Prefetches elements from the dataset for improved performance.
    """
    dataset = tf.data.Dataset.from_tensor_slices((df['review_text'], df['top100']))
    dataset = dataset.batch(batch_size)
    dataset = dataset.map(lambda x, y: (get_bert_embeddings(x), y))
    return dataset.prefetch(tf.data.AUTOTUNE)

----
**7. Prepare the training, validation, and test datasets for training**

In [9]:
train_ds = prepare_dataset(train_df)
val_ds = prepare_dataset(val_df)
test_ds = prepare_dataset(test_df)

----
**8. Define the 1D-CNN model**

In [10]:
kernel_size = 5

def create_rnn_model(input_shape):
    inputs = tf.keras.Input(shape=input_shape)
    x = layers.Conv1D(filters=64, kernel_size=kernel_size, activation='relu', name='1d-cnn_1')(inputs)
    x = layers.MaxPooling1D(pool_size=2, name='maxpool_1')(x)
    x = layers.Conv1D(filters=32, kernel_size=kernel_size, activation='relu', name='1d-cnn_2')(x)
    x = layers.MaxPooling1D(pool_size=2, name='maxpool_2')(x)
    x = layers.GlobalMaxPooling1D(name='globalmaxpool')(x)
    x = layers.Dense(64, activation='relu', name='dense_1')(x)
    outputs = layers.Dense(1, activation='sigmoid')(x)

    model = keras.Model(inputs=inputs, outputs=outputs)
    return model

----
**9. Compile the model**

In [11]:
input_shape = (None, 768)

model = create_rnn_model(input_shape)
model.compile(optimizer='rmsprop',
              loss='binary_crossentropy',
              metrics=['accuracy'])

# Inspect the model summary
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, None, 768)]       0         
                                                                 
 1d-cnn_1 (Conv1D)           (None, None, 64)          245824    
                                                                 
 maxpool_1 (MaxPooling1D)    (None, None, 64)          0         
                                                                 
 1d-cnn_2 (Conv1D)           (None, None, 32)          10272     
                                                                 
 maxpool_2 (MaxPooling1D)    (None, None, 32)          0         
                                                                 
 globalmaxpool (GlobalMaxPo  (None, 32)                0         
 oling1D)                                                        
                                                             

----
**10. Train the model**

In [12]:
# Clear any existing models in memory
tf.keras.backend.clear_session()

# Define the callbacks for the model training
callbacks = [
    tf.keras.callbacks.ModelCheckpoint("EXP_12_1D-CNN_BERT.keras"),
    tf.keras.callbacks.EarlyStopping(monitor='val_accuracy', patience=3, restore_best_weights=True)
]

# Train the model
start_time = time.time()
history = model.fit(train_ds,
                    validation_data=val_ds,
                    validation_steps=len(val_ds),
                    epochs=20,
                    callbacks=callbacks)
end_time = time.time()
training_time = end_time - start_time

# Print the training time
print(f"Training Time: {training_time} seconds")

Epoch 1/20
3/3 [==============================] - 44s 15s/step - loss: 1.0188 - accuracy: 0.5250 - val_loss: 0.7088 - val_accuracy: 0.5000
Epoch 2/20
3/3 [==============================] - 42s 14s/step - loss: 0.6720 - accuracy: 0.4750 - val_loss: 0.6654 - val_accuracy: 0.6500
Epoch 3/20
3/3 [==============================] - 41s 14s/step - loss: 0.5032 - accuracy: 0.9375 - val_loss: 0.9344 - val_accuracy: 0.5000
Epoch 4/20
3/3 [==============================] - 41s 14s/step - loss: 0.5763 - accuracy: 0.6500 - val_loss: 0.5900 - val_accuracy: 0.7000
Epoch 5/20
3/3 [==============================] - 42s 14s/step - loss: 0.3667 - accuracy: 0.7875 - val_loss: 0.6427 - val_accuracy: 0.6500
Epoch 6/20
3/3 [==============================] - 42s 14s/step - loss: 0.1698 - accuracy: 0.9875 - val_loss: 0.5825 - val_accuracy: 0.6000
Epoch 7/20
3/3 [==============================] - 42s 14s/step - loss: 0.1357 - accuracy: 1.0000 - val_loss: 0.5127 - val_accuracy: 0.7000
Training Time: 301.81247687

----
**11. Evaluate the Model**


In [13]:
# Load the best model
model = tf.keras.models.load_model("EXP_12_1D-CNN_BERT.keras")

# Evaluate the model on the test dataset
test_loss, test_accuracy = model.evaluate(test_ds)
print(f"Test Loss: {test_loss}, Test Accuracy: {test_accuracy}")

# Get the training loss, validation loss, training accuracy, and validation accuracy from the history object
training_loss = history.history['loss'][-1] # the -1 index gets the last epoch
validation_loss = history.history['val_loss'][-1]
training_accuracy = history.history['accuracy'][-1]
validation_accuracy = history.history['val_accuracy'][-1]

"""
1.2.2 Extract the training history and add all evaluation metrics into a history DataFrame
"""
# Extract the training history into a pandas DataFrame
history_df = pd.DataFrame({
    'EXP': [12],
    'Model': ['1D-CNN w/ BERT Embeddings'],
    'Training Loss': [training_loss],
    'Training Accuracy': [training_accuracy],
    'Validation Loss': [validation_loss],
    'Validation Accuracy': [validation_accuracy],
    'Test Loss': [test_loss],
    'Test Accuracy': [test_accuracy],
    'Training Time': [training_time]
})

# Inspect the history DataFrame
history_df

1/1 [==============================] - 10s 10s/step - loss: 0.4920 - accuracy: 0.7500
Test Loss: 0.49200958013534546, Test Accuracy: 0.75


,EXP,Model,Training Loss,Training Accuracy,Validation Loss,Validation Accuracy,Test Loss,Test Accuracy,Training Time
0,12,1D-CNN w/ BERT Embeddings,0.135678,1.0,0.512722,0.7,0.49201,0.75,301.812477
